In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDClassifier

In [35]:
# training data
pp = np.load('../Task3/Task3/perfect_patches.npy')
dp = np.load('../Task3/Task3/defect_patches.npy')

# test data
graphene = np.load('../Task3/Task3/full-stack.npy')
plt
# data set of 50/50 perfect to imperfect images
patches = np.concatenate([pp, dp])  # matrices
gt = np.concatenate([np.zeros(len(pp)), np.ones(len(dp))])  # labels
print(gt)

X_train, X_test, y_train, y_test = train_test_split(
    patches,
    gt,
    test_size=0.2,
    shuffle=True,
)

nsamples, nx, ny = X_train.shape
X_train_transformed = X_train.reshape((nsamples, nx * ny))

nsamples, nx, ny = X_test.shape
X_test_transformed = X_test.reshape((nsamples, nx * ny))

[0. 0. 0. ... 1. 1. 1.]


In [36]:
# SVM attempt 1
ratio = len()
clf = OneClassSVM(gamma="auto", nu=ratio)
y_pred = clf.fit_predict(X_train_transformed)
clf.decision_function(X_train_transformed)

TypeError: len() takes exactly one argument (0 given)

In [ ]:
y_train_translated = np.copy(y_train)
y_train_translated[y_train_translated == 1] = -1
y_train_translated[y_train_translated == 0] = 1

y_outliers = y_train_translated[y_train_translated != y_pred]

print(len(y_outliers))
indicies = []
for i, y_val in enumerate(y_pred):
    if y_val != y_train_translated[i]:
        indicies.append(i)

print(indicies)
plt.imshow(X_train[indicies[5]])

In [ ]:


print('Percentage correct: ', 100 * np.sum(y_pred == y_train_translated) / len(y_pred))


In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay, auc
fpr, tpr, thresholds = roc_curve(y_train_translated, y_pred)
roc_auc = auc(fpr, tpr)
roc = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
roc.plot()